In [1]:
import numpy as np 

A = np.array([[1, 2, 3], [4, 5, 6]], dtype=float)
B = np.array([[7, 8], [9, 10], [11, 12]], dtype=float)
A,B

(array([[1., 2., 3.],
        [4., 5., 6.]]),
 array([[ 7.,  8.],
        [ 9., 10.],
        [11., 12.]]))

In [2]:
import taichi as ti
import numpy as np

# Initialize Taichi
ti.init()

# Create a Taichi ndarray
taichi_array = ti.ndarray(ti.f32, shape=(10, 10))

# Perform operations on the Taichi ndarray (if necessary)
taichi_array[3, 4] = 10

# Convert to a NumPy array
numpy_array = taichi_array.to_numpy()
print(numpy_array)


[Taichi] version 1.7.0, llvm 15.0.4, commit 2fd24490, linux, python 3.10.12


[I 12/30/23 22:40:13.860 2753] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


[Taichi] Starting on arch=x64
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. 10.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


In [3]:
import taichi as ti
import numpy as np
ti.init()

a = np.zeros((5, 5))

@ti.kernel
def test(a: ti.types.ndarray()):
    for i in range(a.shape[0]):  # a parallel for loop
        for j in range(a.shape[1]):
            a[i, j] = i + j

test(a)
print(a)

[Taichi] Starting on arch=x64
[[0. 1. 2. 3. 4.]
 [1. 2. 3. 4. 5.]
 [2. 3. 4. 5. 6.]
 [3. 4. 5. 6. 7.]
 [4. 5. 6. 7. 8.]]


In [4]:
import taichi as ti
import numpy as np

ti.init(arch=ti.cpu)  # Use CPU by default

def matrix_multiply_numpy_to_taichi(A: np.ndarray, B: np.ndarray) -> np.ndarray:
    C = ti.field(shape=(A.shape[0], B.shape[1]), dtype=ti.f64)

    # Convert the numpy arrays to Taichi ndarrays
    A_ti = ti.field(shape=A.shape, dtype=ti.f64)
    B_ti = ti.field(shape=B.shape, dtype=ti.f64)
    A_ti.from_numpy(A)
    B_ti.from_numpy(B)
    sum = ti.field(dtype=ti.f64, shape=())

    @ti.kernel
    def _taichi_compute():
        for i in range(A.shape[0]):
            for j in range(B.shape[1]):
                sum[None] = 0.0
                for k in range(A.shape[1]):
                    sum[None] += A_ti[i, k] * B_ti[k, j]
                C[i,j] = sum[None]

    _taichi_compute()
    return C.to_numpy()



[Taichi] Starting on arch=x64


In [5]:
A, B

(array([[1., 2., 3.],
        [4., 5., 6.]]),
 array([[ 7.,  8.],
        [ 9., 10.],
        [11., 12.]]))

In [6]:
%%time
# Example usage
C = matrix_multiply_numpy_to_taichi(A, B)
C


CPU times: user 71.6 ms, sys: 3.33 ms, total: 74.9 ms
Wall time: 72.7 ms


array([[ 58.,  64.],
       [139., 154.]])

In [7]:
A1 = np.random.randn(20, 30)
B1 = np.random.randn(30, 40)
A1.shape, B1.shape

((20, 30), (30, 40))

In [8]:
%%time
# Example usage
C1 = matrix_multiply_numpy_to_taichi(A1, B1)
C1.shape


CPU times: user 73.9 ms, sys: 144 µs, total: 74 ms
Wall time: 71.2 ms


(20, 40)

In [9]:
import taichi as ti

ti.init(arch=ti.cpu)  # Use CPU by default

def matrix_multiply_taichi(A: np.ndarray, B: np.ndarray) -> np.ndarray:
    C = ti.field(shape=(A.shape[0], B.shape[1]), dtype=ti.f64)

    # Convert the numpy arrays to Taichi ndarrays
    A_ti = ti.field(shape=A.shape, dtype=ti.f64)
    B_ti = ti.field(shape=B.shape, dtype=ti.f64)
    A_ti.from_numpy(A)
    B_ti.from_numpy(B)
    sum = ti.field(dtype=ti.f64, shape=())

    @ti.kernel
    def _taichi_compute():
        ti.loop_config(serialize=True)
        for i in range(A.shape[0]):
            for j in range(B.shape[1]):
                sum[None] = 0.0
                for k in range(A.shape[1]):
                    sum[None] += A_ti[i, k] * B_ti[k, j]
                C[i,j] = sum[None]

    _taichi_compute()
    return C.to_numpy()


[Taichi] Starting on arch=x64


In [10]:
# list to collect run-time metrics
# initialize with results for native C implementation
test_results = [
    {"function": "native_c", "duration": 0.009, "results": None}
]

DIM_SIZE = 200
np.random.seed(0)
# Create two random square matrices
A = np.random.rand(DIM_SIZE, DIM_SIZE)
B = np.random.rand(DIM_SIZE, DIM_SIZE)
A.shape, B.shape


((200, 200), (200, 200))

In [20]:
%%time
result = matrix_multiply_taichi(A, B)

CPU times: user 229 ms, sys: 1.89 ms, total: 231 ms
Wall time: 227 ms


In [31]:
def matrix_multiply_taichi2(A: np.ndarray, B: np.ndarray) -> np.ndarray:
    C = ti.field(shape=(A.shape[0], B.shape[1]), dtype=ti.f64)

    # Convert the numpy arrays to Taichi ndarrays
    A_ti = ti.field(shape=A.shape, dtype=ti.f64)
    B_ti = ti.field(shape=B.shape, dtype=ti.f64)
    # A_ti.from_numpy(A)
    # B_ti.from_numpy(B)
    sum = ti.field(dtype=ti.f64, shape=())

    @ti.kernel
    def _taichi_compute():
        for i in range(A.shape[0]):
            ti.static_print(f"type(i)={i}")
            for j in range(B.shape[1]):
                sum[None] = 0.0
                for k in range(A.shape[1]):
                    sum[None] += A[i, k] * B[k, j]
                C[i,j] = sum[None]

    _taichi_compute()
    return C.to_numpy()

In [32]:
%%time
result = matrix_multiply_taichi2(A, B)

['__ti_format__', 'type(i)=', <ti.Expr>, '']


TaichiCompilationError: 
File "/tmp/ipykernel_2753/2406462612.py", line 18, in _taichi_compute:
                    sum[None] += A[i, k] * B[k, j]
                                 ^^^^^^^
Traceback (most recent call last):
  File "/home/vscode/.local/lib/python3.10/site-packages/taichi/lang/ast/ast_transformer_utils.py", line 27, in __call__
    return method(ctx, node)
  File "/home/vscode/.local/lib/python3.10/site-packages/taichi/lang/ast/ast_transformer.py", line 240, in build_Subscript
    node.ptr = impl.subscript(ctx.ast_builder, node.value.ptr, *node.slice.ptr)
  File "/home/vscode/.local/lib/python3.10/site-packages/taichi/lang/util.py", line 325, in wrapped
    return func(*args, **kwargs)
  File "/home/vscode/.local/lib/python3.10/site-packages/taichi/lang/impl.py", line 206, in subscript
    return value.__getitem__(_indices)
IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
